## Data sourcing

Source data from various source systems and ingest them using python code.

1. Parquet files
2. CSV files
3. APIs
4. RDBMS databases
5. HTML

In [ ]:
# import modules
import certifi
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import urllib3
from urllib3 import request
from unicodedata import normalize
import os
import http

### Sourcing Parquet data

Please visit the url https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
# check current working directory
print(os.getcwd())

In [ ]:
# Read data from the Parquet file. We use pandas read_parquet method for ease and speed.
df_parquet = pd.read_parquet("yellow_tripdata_2022-01.parquet")
#get row count of df
print(df_parquet.shape[0])
#get column count of df
print(df_parquet.shape[1])
#get size on disk of file
print(os.path.getsize("yellow_tripdata_2022-01.parquet"))


In [ ]:
df_parquet.head()

In [ ]:
#show average "trip_distance", average "fare_amount", and average "tip_amount" from df columns
print(f"Average trip distance: {df_parquet['trip_distance'].mean()} ")
print(f"Average fare amount: {df_parquet['fare_amount'].mean()} ")
print(f"Average tip amount: {df_parquet['tip_amount'].mean()} ")

### Sourcing CSV data 

Please visit the url https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=500


In [ ]:
# Read data from the CSV file. We use pandas read_csv method for ease and speed.
df_csv = pd.read_csv("h9gi-nx95.csv")
df_csv.head()
#get row count of df
print(df_csv.shape[0])
#get size on disk of file
print(os.path.getsize("h9gi-nx95.csv"))
#show total number of crash counts grouped by column "contributing_factor_vehicle_1"
df_csv['contributing_factor_vehicle_1'].value_counts()

### Sourcing data from APIs

Please make sure to install the certifi library using - pipenv install certifi

In [ ]:
# get api data from url
url = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=500'

# Check if API is available to retrive the data
apt_status = request('GET', url).status # removed "http." from in front of request
print(apt_status)
if apt_status == 200:
    # Sometimes we get certificate error . We should never silence this error as this may cause a security threat.
    # Create a Pool manager that can be used to read the API response 
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    data = json.loads(http.request('GET', url).data.decode('utf-8'))
    df_api = pd.json_normalize(data)
else:
    df_api = pd.Dataframe()
df_api.head(10)




In [ ]:
#print row count of df_api
print(df_api.shape[0])


In [ ]:

#convert column "crash_time" to hour of day
print("Crash Time as datetime:")
df_api['crash_time'] = pd.to_datetime(df_api['crash_time'])

#print new list of crash_time values in new datetime format, showing the hour of day and am/pm
print("Crash Time as datetime formatted as hour (am/pm):")
print(df_api['crash_time'].dt.strftime('%I %p'))

#print count of records grouped by "crash_date"
print("Count of records grouped by crash_date:")
print(df_api['crash_date'].value_counts())

#print the count of records grouped by crash_time as hour of day, sorted by sum count descending
print("Count of records grouped by crash_time as hour of day:")
#add headers to the output
print("Hour of Day: Count")
print(df_api['crash_time'].dt.hour.value_counts().sort_values(ascending=False))

#convert the crash_time to seconds from midnight 
seconds_since_midnight = df_api['crash_time'].dt.hour*3600 + df_api['crash_time'].dt.minute*60 + df_api['crash_time'].dt.second
#calculate the average crash time in seconds
average_crash_time = seconds_since_midnight.mean()
#convert avg seconds back to HH:MM:SS format
average_crash_time = pd.to_datetime(average_crash_time, unit='s').strftime('%H:%M:%S')
print(f"The average crash time is: {average_crash_time}")


### Sourcing Data from RDBMS tables

In [ ]:
# # Read sqlite query results into a pandas DataFrame
# with sqlite3.connect("movies.sqlite") as conn:
#     df = pd.read_sql("SELECT * from movies", conn)
# df.head()

In [ ]:
#test using pyodbc
import pyodbc

#import pyodbc
#connect to sql database
with pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=datadevjobs.d.db.vu.local;DATABASE=DataDevJobsRepo;Trusted_Connection=yes;') as conn:
    query = "SELECT * FROM Logging.SolutionOverviewServers"
    df_sql = pd.read_sql(query, conn)
df_sql.head()

# with pyodbc, open connection to sql database "datadevjobs.d.db.vu.local"
    #read the db "DataDevJobsRepo" and table "Logging.SolutionOverviewServers" into a pandas dataframe

In [ ]:
#test using sqlalchemy
from sqlalchemy import create_engine

#connect to sql database
engine = create_engine('mssql+pyodbc://datadevjobs.d.db.vu.local/DataDevJobsRepo?driver=ODBC+Driver+17+for+SQL+Server')
with engine.connect() as conn:
    df_sqlalchemy = pd.read_sql("SELECT * FROM Logging.SolutionOverviewServers", conn)
df_sqlalchemy.head()

# Sourcing data from Webpages

Please visit the url https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)

In [26]:
# get data from url
df_html = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)',match = 'by country') # the match parameter filters results to only tables with a specific string in the html "caption" tag
# Let's see how many tables are there with tag ' by county'
print(len(df_html)) # There are 4 tables
# Let's see the first table
df_html[0]

4


Country/Territory UN region IMF[1][13]            World Bank[14]  \
    Country/Territory UN region   Forecast       Year       Estimate   
0               World         —  104476432       2023      100562011   
1       United States  Americas   26949643       2023       25462700   
2               China      Asia   17700899  2023[n 1]       17963171   
3             Germany    Europe    4429838       2023        4072192   
4               Japan      Asia    4230862       2023        4231141   
..                ...       ...        ...        ...            ...   
208             Palau   Oceania        267       2023              —   
209          Kiribati   Oceania        246       2023            223   
210             Nauru   Oceania        150       2023            151   
211        Montserrat  Americas          —          —              —   
212            Tuvalu   Oceania         63       2023             60   

               United Nations[15]             
          Year           Estimate       Year  
0         2022           96698005       2021  
1         2022           23315081       2021  
2    2022[n 3]           17734131  2021[n 1]  
3         2022            4259935       2021  
4         2022            4940878       2021  
..         ...                ...        ...  
208          —                218       2021  
209       2022                227       2021  
210       2022                155       2021  
211          —                 72       2021  
212       2022                 60       2021  

[213 rows x 8 columns]